In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [51]:
#Cargamos la base de datos 'dataset_filtrado'
df_filtered = pd.read_csv("C:/Users/guard/OneDrive/Desktop/Henry Data Science/Proyecto MLOps/datasets/dataset_filtrado_15000.csv", sep=",")

df_filtered.head(2)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western,genres_1,cast_names,directing_names
0,0.0,76600,en,6.020055,2020-12-16,0.0,0.0,Avatar 2,0.0,58.0,...,0,0,0,0,0,0,0,"Action, Adventure, Fantasy, Science FictionAct...",Sin Datos,Sin Datos
1,12000000.0,299782,en,0.238154,2018-12-31,0.0,0.0,The Other Side of the Wind,0.0,1.0,...,0,0,0,0,0,0,0,"Comedy, DramaComedy, Drama",Sin Datos,Sin Datos


In [52]:
# Preprocesamiento de Datos - Utilizando 'title' para la recomendación
df_filtered['title'] = df_filtered['title'].fillna('')
# Vectorización de Textos - Usar TfidfVectorizer para transformar los títulos en vectores
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_filtered['title'])
# Cálculo de Similaridad - Utilizar la similaridad del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [53]:
# Crear una serie con los títulos de las películas como índice
indices = pd.Series(df_filtered.index, index=df_filtered['title']).drop_duplicates()

In [54]:
def get_recommendations(title, cosine_sim, df_filtered):
    # Normalizar el título
    title = title.strip().lower()  # Eliminar espacios y convertir a minúsculas
    
    # Obtener el índice de la película que coincide con el título
    matching_movies = df_filtered[df_filtered['title'].str.lower().str.contains(title)]
    
    if matching_movies.empty:
        raise ValueError(f"No se encontró ninguna película con el título: '{title}'")

    idx = matching_movies.index[0]
    
    # Obtener las puntuaciones de similaridad para esa película
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas basadas en las puntuaciones de similaridad
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las puntuaciones de las 5 películas más similares
    sim_scores = sim_scores[1:6]  # Cambiado a 5

    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]

    # Devolver los títulos de las películas recomendadas
    return df_filtered['title'].iloc[movie_indices].tolist()


print(get_recommendations('Toy Story', cosine_sim, df_filtered))

['Toy Story 3', 'Toy Story 2', 'Toy Story of Terror!', 'Toy Reanimator', 'One More Time']
